## xTB via commandline

https://xtb-docs.readthedocs.io/en/latest/contents.html

### Optimization


Running a simple xtb optimization via command line

```console
xtb glutamine.xyz
```

# Input file can have many format (tmol,xyz,mol,sdf,pdb)
# --ohess optimize then frequency calculation
# -c charge
# --alpb implicit solvation model (Analytical linearized Poisson-Boltzmann)
# --uhf number of unpaired electrons if not 0

In [ ]:
# in the main output you can find orbital energies and occupation
# the optimzed structure is in xtbopt.xyz

! cat xtbopt.xyz

In [ ]:
# Let's look at some of the output files

! cat charges

In [ ]:
! cat wbo

In [ ]:
! cat vibspectrum

In [ ]:
# We can get more printouts by changing default arguments in the $write part of the input

text = "$write\n"
text += "mulliken=true\n"              # Mulliken population analysis
text += "density=true\n"               # Density in cube format
text += "spin population=true\n"       # Spin population
text += "esp=true\n"                   # Electrostatic potential
text += "output file=properties.out\n" # Redirect property printout
text += "json=true\n"                  # Write json file with various info
text += "$end"

#open("xtb-printouts.inp", "w")
with open("xtb-printouts.inp","w") as f:
    f.write(text)

In [ ]:
text = """
$write
mulliken=true             
density=true             
spin population=true       
esp=true                  
output file=properties.out 
json=true                  
$end
""".lstrip()

In [ ]:
! xtb xtbopt.xyz --input xtb-printouts.inp

In [ ]:
# Some goodies included in the xTB package
# Global Electrophilicity index

! xtb xtbopt.xyz --vomega

In [ ]:
# Fukui indices

! xtb xtbopt.xyz --vfukui

In [ ]:
# Vertical ionization potential and ionization energy

! xtb xtbopt.xyz --vipea

### Constrained optimization

In [ ]:
# Example for a dihedral scan
# Generate file containing constrains

text = "$constrain\n"
text += "force constant=0.05\n"
text += "$scan\n"
text += "dihedral: 15,14,13,1,0.0;0.0,180,90\n"
text += "$end"

open("scan-1.inp", "w")
with open("scan-1.inp","a") as f:
    f.write(text)

In [ ]:
! xtb xtbopt.xyz --opt --input scan-1.inp >> scan-1.out

In [ ]:
! cat xtbscan.log

In [ ]:
# Example for a 2D bond length scan
# For concerted scan number of steps has to match

text = "$constrain\n"
text += "force constant=0.05\n"
text += "$scan\n"
text += "mode=concerted\n"
text += "distance: 1,4,0.5;0.5,1.4,50\n"
text += "distance: 3,5,0.5;0.5,1.4,50\n"
text += "$end"

open("scan-2.inp", "w")
with open("scan-2.inp","a") as f:
    f.write(text)

## CREST

https://crest-lab.github.io/crest-docs/

### Generating an ensemble

In [ ]:
# Run a conformer search with Crest.
# --noreftopo prevent a lot of initial crashes by not doing a topology check prior to the conformational search.

! crest xtbopt.xyz --gfn2 --alpb water --noreftopo --T 24

# --gfn2//gfnff this does sampling and opt with GFN-FF and singlepoint with GFN2-xTB for greater speeds.

In [ ]:
# crest_best.xyz contains the most stable structure

! cat crest_best.xyz

In [ ]:
# crest_conformers.xyz contains the conformer ensemble
# crest.energies contains the relative energies of the ensemble

! cat crest_conformers.xyz

In [ ]:
! mv crest_conformers.xyz glutamine_conformers.xyz

### Adding constrains

In [ ]:
# It is very common for crest to fragment/rearange weaker bonds, you might need to add constrains.

In [ ]:
# TS conformers search can be performed in this way by constraining the TS mode. 

In [ ]:
# Example of a constrains file for CREST

text = "$constrain\n"
text += "force constant=0.05\n"
text += "distance: 1,2,1.85\n"
text += "$end"

open("CREST-1.inp", "w")
with open("CREST-1.inp","a") as f:
    f.write(text)

In [ ]:
! crest xtbopt.xyz --cinp CREST-1.inp --gfn2 --alpb water --noreftopo --T 24 >> CREST-1.out

In [ ]:
# Some usefull preset constrain arguments

# --cbonds will constraints all covalent bonds
# --cmetal constraints all M-X bonds
# --cheavy constraints all heavy atom bonds

In [ ]:
# If constraining a large portion of the molecule it is recomanded to constrain atom this way instead:
# This for example will constrain atom 1 to 26 and only perform sampling on atom 27 to 41

text = "$constrain\n"
text += "force constant=0.05\n"
text += "atoms: 1-26\n"
text += "$metadyn"
text += "atoms: 27-41"
text += "$end"

open("CREST-2.inp", "w")
with open("CREST-2.inp","a") as f:
    f.write(text)

In [ ]:
# We can also generate the constrain input file automatically with:

# ! crest xtbopt.xyz --constrain 1-26 --gfn2 --alpb water --noreftopo --T 24

### Protonation/deprotonation module

In [ ]:
# Using the protonate tool to generate optimized protomers (no conf search)

! crest xtbopt.xyz --protonate --T 24 >> protonate.out
! cat protonated.xyz

In [ ]:
# You can specify another type of monoatomic ion (default is proton)
# For example use --swel ca2+ for dicationic calcium

! crest xtbopt.xyz --swel Na1+ --protonate --T 24 >> NaProtonate.out

In [ ]:
! crest xtbopt.xyz --deprotonate --T 24 >> deprotonate.out

In [ ]:
! crest xtbopt.xyz --tautomerize --T 24 >> tautomerize.out

### Non-covalent system

In [ ]:
! crest waters.xyz --nci --gfn2 --alpb water --noreftopo --T 24 >> crest-water.out

In [ ]:
! mv crest_conformers.xyz waters_conformers.xyz

In [ ]:
# Adds an ellipsoid wall potential around the system during the MTD sampling to prevent dissociation and then removed during optimization
# can scale with -wscal

In [ ]:
! crest protonated-Na.xyz --nci --gfn2 --alpb water --noreftopo --T 24 >> crest_glutamine-Na.out

In [ ]:
! mv crest_conformers.xyz glutamine-Na_conformers.xyz

### Sorting ensembles

In [ ]:
# A ensemble of conformers can be refined using the cregen function of crest

! crest xtbopt.xyz --cregen glutamine_conformers.xyz --ewin 3 

In [ ]:
# --ewin (default 6kcal/mol)
# --rthr RMSD treshold (default 0.125 Angstrom)
# --ethr energy treshold between conformer pairs (default 0.05 kcal/mol)
# --nmr --eqv NMR equivalencies
# --temp (default is 298.15 K)
# --notopo disables topology checks

In [ ]:
# This provides us with a sorted and filtered ensemble in crest_ensemble.xyz

! cat crest_ensemble.xyz

In [ ]:
# Cregen can also cluster structures based on diherdral angles

! crest xtbopt.xyz --cregen glutamine_conformers.xyz --cluster

## ORCA-xTB implementation

https://www.orcasoftware.de/tutorials_orca/multi/basics.html

### Ground state optimization

In [ ]:
# A simple ORCA GS optimization using GFN2-XTB

text = "! pal8 GFN2-XTB opt NumFreq alpb(ch2cl2)\n"
text += "%geom\n"
text += "end\n"
text += "*xyz 0 1\n"

open("ORCA_GS.inp", "w")
with open("ORCA_GS.inp","a") as f:
    f.write(text)

In [ ]:
! orca ORCA_OPT.inp > ORCA_OPT.out

### Transition state optimization

In [ ]:
# A simple ORCA TS optimization using GFN2-XTB

text = "! pal8 GFN2-XTB optTS NumFreq alpb(ch2cl2)\n"
text += "%geom\n"
text += "end\n"
text += "*xyz 0 1\n"

open("ORCA_TS.inp", "w")
with open("ORCA_TS.inp","a") as f:
    f.write(text)

In [ ]:
! orca ORCA_TS.inp > ORCA_TS.out

### DFT/xTB multiscale calculations

In [ ]:
# A simple ORCA optimization using GFN2-XTB
# Atoms not specified in the QMATOMS list will be ran using xTB

text = "! pal8 QM/XTB opt B3LYP DEF2-TZVPP\n"
text += "%QMMM QMATOMS {1:5,7:9}\n"
text += "end\n"
text += "end\n"
text += "%geom\n"
text += "end\n"
text += "*xyz 0 1\n"

open("ORCA_QM-XTB.inp", "w")
with open("ORCA_QM-XTB.inp","a") as f:
    f.write(text)

In [ ]:
! orca ORCA_QM-XTB.inp > ORCA_QM-XTB.out